# Assignment 1: Dynamic Programming

---

## Task 1) Edit Distances

Implement the [Hamming](https://en.wikipedia.org/wiki/Hamming_distance) and [Levenshtein](https://en.wikipedia.org/wiki/Levenshtein_distance) (edit) distances and compute them for the for the following word pairs.
It may help to compute them by hand first.

<img src = "./assets/97090.jpg" width="33%" /> <img src = "./assets/97222.jpg" width="33%" /> <img src = "./assets/97669.jpg" width="33%" />

Aside from computing the distance (which is a scalar), do the backtrace and compute the edit transcript (and thus alignment).

---

In [15]:
WORD_PAIRS = [
    ("GCGTATGAGGCTAACGC", "GCTATGCGGCTATACGC"),
    ("kühler schrank", "schüler krank"),
    ("the longest", "longest day"),
    ("nicht ausgeloggt", "licht ausgenockt"),
    ("gurken schaben", "schurkengaben")
]

In [16]:
def align(s1: str, s2: str) -> tuple[str, str]:
    """"
    Align two strings by padding the shorter string with spaces.
    """
    max_len = max(len(s1), len(s2))
    return (s1.ljust(max_len), s2.ljust(max_len))

In [17]:
def hamming(s1: str, s2: str) -> int:
    """
    Compute the hamming distance between two strings.

    Arguments:
    s1: First string of word pair.
    s2: Second string of word pair.

    Returns the distance as int.
    """
    # Hint: Think about how you can deal with unequal word lengths.
    
    ### YOUR CODE HERE
    s1, s2 = align(s1, s2)

    d = 0

    for i in range(len(s1)):
        if s1[i] != s2[i]:
            d += 1
    
    return d
    ### END YOUR CODE

In [18]:
for wordpair in WORD_PAIRS:
    dist = hamming(s1=wordpair[0], s2=wordpair[1])
    print("hamming('{}', '{}') = {}".format(
        wordpair[0], wordpair[1], dist
    ))

### EXPECTED
# hamming('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 10
# hamming('kühler schrank', 'schüler krank') = 13
# hamming('the longest', 'longest day') = 11
# hamming('nicht ausgeloggt', 'licht ausgenockt') = 4
# hamming('gurken schaben', 'schurkengaben') = 14

hamming('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 10
hamming('kühler schrank', 'schüler krank') = 13
hamming('the longest', 'longest day') = 11
hamming('nicht ausgeloggt', 'licht ausgenockt') = 4
hamming('gurken schaben', 'schurkengaben') = 14


In [19]:
import numpy as np

In [20]:
def levenshtein(s1: str, s2: str, cost={'m': 0, 's': 1, 'i': 1, 'd': 1}) -> tuple[int, str]:
    """
    Compute the levenshtein (edit) distance between two strings.

    Arguments:
    s1: First string of word pair.
    s2: Second string of word pair.

    Returns the distance as int and edit transcript as string.
    """
    # Hint: The probably most intuitive approach is bottom-up.
    
    ### YOUR CODE HERE
    x, y = len(s1), len(s2)
    m = ''
    D = np.zeros(shape=(x + 1, y + 1), dtype = int)
    D[0, 1:] = range(1, y + 1)
    D[1:, 0] = range(1, x + 1)
    
    # Fill the table
    for i in range(1, x + 1):
        for j in range(1, y + 1):
            d = cost['m'] if s1[i - 1] == s2[j - 1] else cost['s']
            u = D[i - 1, j] + cost['d']
            l = D[i, j - 1] + cost['i']
            ul = D[i - 1, j - 1] + d
            D[i, j] = min(u, l, ul)

    # Write the transcript
    i, j = x, y
    while i > 0 or j > 0:
        c = D[i, j]
        if i > 0 and c == D[i - 1, j] + cost['d']:
            m += 'd'
            i, j = i - 1, j     
        elif j > 0 and c == D[i, j - 1] + cost['i']:
            m += 'i'
            i, j = i, j - 1
        else:
            if c == D[i - 1, j - 1] + cost['s'] and s1[i - 1] != s2[j - 1]:
                m += 's'
            else:
                m += 'm'
            i, j = i - 1, j - 1

    return D[-1, -1], m[::-1]

    ### END YOUR CODE

In [21]:
for wordpair in WORD_PAIRS:
    dist, transcript = levenshtein(s1=wordpair[0], s2=wordpair[1])
    print("levenshtein('{}', '{}') = {} ({})".format(
        wordpair[0], wordpair[1], dist, transcript
    ))

### EXPECTED
# levenshtein('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 3 (mmdmmmmsmmmmmimmmm)
# levenshtein('kühler schrank', 'schüler krank') = 6 (ssmimmmmsddmmmm)
# levenshtein('the longest', 'longest day') = 8 (ddddmmmmmmmiiii)
# levenshtein('nicht ausgeloggt', 'licht ausgenockt') = 4 (smmmmmmmmmmsmssm)
# levenshtein('gurken schaben', 'schurkengaben') = 7 (siimmmmmsdddmmmm)

levenshtein('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 3 (mmdmmmmsmmmmmimmmm)
levenshtein('kühler schrank', 'schüler krank') = 6 (ssmimmmmsddmmmm)
levenshtein('the longest', 'longest day') = 8 (ddddmmmmmmmiiii)
levenshtein('nicht ausgeloggt', 'licht ausgenockt') = 4 (smmmmmmmmmmsmssm)
levenshtein('gurken schaben', 'schurkengaben') = 7 (siimmmmmsdddmmmm)


---

## Task 2) Basic Spelling Correction using Edit Distance

For spelling correction, we will use prior knowledge, to put _some_ learning into our system.

The underlying idea is the _Noisy Channel Model_, that is: The user _intends_ to write a word `w`, but through some noise in the process, happens to type the word `x`.

The correct word $\hat{w}$ is that word, that is a valid candidate and has the highest probability:

$$
\begin{eqnarray}
\DeclareMathOperator*{\argmax}{argmax}
\hat{w} & = & \argmax_{w \in V} P(w | x) \\
        & = & \argmax_{w \in V} \frac{P(x|w) P(w)}{P(x)} \\
        & = & \argmax_{w \in V} P(x|w) P(w)
\end{eqnarray}
$$

1. The candidates $V$ can be obtained from a _vocabulary_.
2. The probability $P(w)$ of a word $w$ can be _learned (counted) from data_.
3. The probability $P(x|w)$ is more complicated... It could be learned from data, but we could also use a _heuristic_ that relates to the edit distance, e.g. rank by distance.

You may use [Peter Norvig's count_1w.txt](http://norvig.com/ngrams/) file, [local mirror](res/count_1w.tar.bz2).
Note that it may help to restrict to the first 10k words to get started.

---

In [22]:
EXAMPLES = [
    "pirates",    # in-voc
    "pirutes",    # pirates?
    "continoisly",  # continuosly?
]

In [23]:
import pandas as pd

In [24]:
### TODO: Prepare the vocabulary

### YOUR CODE HERE
V = pd.read_csv('data/count_1w.txt', delimiter='\t', header=None, names=['word', 'counting'], dtype={'word': str, 'counting': int})
V = V.sort_values(by='counting', ascending=False).head(100000)
V = V.drop('counting', axis=1)
### END YOUR CODE

In [25]:
def suggest(w: str, dist_fn, max_cand=5) -> list[tuple[str, int, int]]:
    """
    Compute suggestions for spelling correction using edit distance.
    
    Arguments:
    w: Word in question.
    dist_fn: Distance function to use (e.g. levenshtein).
    max_cand: Maximum number of suggestions.

    Returns a list of tuples (word, dist, score) sorted by score and distance.
    """
    ### YOUR CODE HERE
    df = V.copy()
    df['dist'] = df['word'].apply((lambda x: dist_fn(w, str(x))[0]))
    df.sort_values(by='dist', inplace=True)
    return [tuple(r) for r in df.head(max_cand).to_numpy()]
    ### END YOUR CODE

In [26]:
# How does your suggest function behave with levenshtein distance?

for word in EXAMPLES:
    suggestions = suggest(w=word, dist_fn=levenshtein, max_cand=3)
    print("{} {}".format(word, suggestions))

### EXPECTED
### Notice: your scores may vary!
# pirates [('pirates', 0, -11.408058827802126)]
# pirutes [('pirates', 1, -11.408058827802126), ('minutes', 2, -8.717825438953103), ('viruses', 2, -11.111468702571859)]
# continoisly [('continously', 1, -15.735337826575178), ('continuously', 2, -11.560071979871001), ('continuosly', 2, -17.009283000138204)]

pirates [('pirates', 0), ('pilates', 1), ('pirated', 1)]
pirutes [('pirates', 1), ('prunes', 2), ('picutres', 2)]
continoisly [('continuously', 2), ('continuity', 3), ('continous', 3)]


### Food for Thought

- How would you modify the implementation so that it works fast for large lexica (eg. the full file above)?
- How would you modify the implementation so that it works "while typing" instead of at the end of a word?
- How do you handle unknown/new words?

---

## Task 3) Needleman-Wunsch: Keyboard-aware Auto-Correct

In the parts 1 and 2 above, we applied uniform cost to all substitutions.
This does not really make sense if you look at a keyboard: the QWERTY layout will favor certain substitutions (eg. _a_ and _s_), while others are fairly unlikely (eg. _a_ and _k_).

Implement the [Needleman-Wunsch algorithm](https://en.wikipedia.org/wiki/Needleman–Wunsch_algorithm) which is very similar to the [Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance), however it doesn't _minimize the cost_ but _maximizes the similarity_.
For a good measure of similarity, implement a metric that computes a meaningful weight for a given character substitution.

---

In [27]:
def keyboardsim(s1: str, s2: str) -> float:
    ### YOUR CODE HERE
    
    raise NotImplementedError()
    
    ### END YOUR CODE


def nw(s1: str, s2: str, d: float, sim_fn) -> float:
    """
    Apply needleman-wunsch algorithm.
    
    Arguments:
    s1: First string of word pair.
    s2: Second string of word pair.
    d: Gap penalty score.
    sim_fn: Similarity function to use.

    Returns the score as float.
    """
    ### YOUR CODE HERE
    
    raise NotImplementedError()
    
    ### END YOUR CODE


def nw_based_dist(s1: str, s2: str) -> (int, str):
    """
    Compute the needleman-wunsch distance between two strings.
    
    Arguments:
    s1: First string of word pair.
    s2: Second string of word pair.
    
    Returns the distance as int and <unsupported> string.
    """
    ### YOUR CODE HERE
    
    raise NotImplementedError()
    # Hint: return dist, "<unsupported>"
    
    ### END YOUR CODE

In [28]:
# How does your suggest function behave with nw and a keyboard-aware similarity?

for word in EXAMPLES:
    suggestions = suggest(w=word, dist_fn=nw_based_dist, max_cand=3)
    print("{} {}".format(word, suggestions))

NotImplementedError: 

### Food for Thought

- What could be better heuristics for similarity resp. cost of substitution than the one above?
- What about capitalization, punctiation and special characters?
- What about swipe-to-type?